In [48]:
import numpy as np
import pandas as pd
import geopandas as gpd
import psrcelmerpy

In [49]:
eg_conn = psrcelmerpy.ElmerGeoConn()

# Load TransRefEdges
gdf = eg_conn.read_geolayer('transRefEdges')
gdf = gdf.to_crs('EPSG:2285')

In [50]:
# Get bicycle attribute tables
e_conn = psrcelmerpy.ElmerConn()
mode_att = eg_conn.get_table(schema='dbo', table_name='modeattributes')

# gdf['PSRCEdgeID']
gdf = gdf.merge(mode_att, on='PSRCEdgeID', how='left')


In [51]:
mode_att.columns
# IJBikeFacilities is for completeness do NOT use IJBikeFacility!
# IJPedFacilities
# mode_att['IJBikeFacilities']
# Use IJBikeType
mode_att['IJBikeType'].value_counts()

# For Ped/Bike Facilities
# -1 partial
# 0 no facilities
# 1 complete facilties
# 1 or -1 for sidewalks and bikes
mode_att['IJPedFacilities'].value_counts()

 0.0    174888
 1.0     71148
-1.0       937
Name: IJPedFacilities, dtype: int64

In [52]:
# Load bike stress table, adapted from Lowry
# https://www1.coe.neu.edu/~pfurth/Furth%20papers/2016%20Prioritizing%20to%20improve%20low-stress%20network%20connectivity%20Lowry,%20Furth,%20Hadden-Loh.pdf

df_data = pd.read_csv('bike_stress_table.csv')

In [66]:
# Reformat speed limit and lanes to match the bike stress data
# Creating a new column for speed limits where minimum is capped at 25 and max at 35
# for dir in ['IJ','JI']:
#     gdf[dir+'SpeedLimit_new'] = gdf[dir+'SpeedLimit'].copy()
#     gdf.loc[gdf[dir+'SpeedLimit'] <= 25, dir+'SpeedLimit_new'] = 25
#     gdf.loc[gdf[dir+'SpeedLimit'] >= 35, dir+'SpeedLimit_new'] = 35

#     # Set maximum lanes at 6 and minimum at 2

#     gdf[dir+'lanes_new'] = gdf[dir+'LanesGPAM'].copy().astype('int')
#     gdf.loc[gdf[dir+'LanesGPAM'] >= 6, dir+'lanes_new'] = 6
#     gdf.loc[gdf[dir+'LanesGPAM'] <2, dir+'lanes_new'] = 2

#     # Lowry has different values for neighborhood street for each bike facility type
#     # Create a flag for an local street
#     gdf[dir+'LocalStreet'] = 0
#     gdf.loc[gdf['FacilityType'] == 9, 'LocalStreet'] = 1


# The Lowry table provides values based on bi-directional data of lanes and speed limit
# The transRefEdges network is made up of directional links, so we need to sum lanes to match Lowry

# Assume speed limit will be the same in both directions, use IJ direction
gdf['SpeedLimit_new'] = gdf['IJSpeedLimit'].copy()
gdf.loc[gdf['IJSpeedLimit'] <= 25, 'SpeedLimit_new'] = 25
gdf.loc[gdf['IJSpeedLimit'] >= 35, 'SpeedLimit_new'] = 35

# Lanes will be the sum of both directions
gdf['lanes_new'] = (gdf['IJLanesGPAM']+gdf['JILanesGPAM']).astype('int')

# Set maximum lanes at 6 and minimum at 2
gdf.loc[gdf['lanes_new'] >= 6, 'lanes_new'] = 6
gdf.loc[gdf['lanes_new'] <2, 'lanes_new'] = 2

# Lowry has different values for neighborhood street for each bike facility type
# Create a flag for an local street
gdf['LocalStreet'] = 0
gdf.loc[gdf['FacilityType'] == 9, 'LocalStreet'] = 1

In [67]:
gdf['FacilityType'] = gdf['FacilityType'].astype('int')
gdf['FacilityType'].unique()

# Remove centroid connectors and freeways
facility_type_lookup = {
    1:'Freeway',   # Interstate
    2:'Freeway',   # Other Freeway
    3:'Freeway', # Expressway
    4:'Ramp',
    5:'Arterial',    # Principal arterial
    6:'Arterial',    # Minor Arterial
    7:'Collector',    # Major Collector
    8:'Collector',    # Minor Collector
    9:'Collector',   # Local
    10:'Busway',
    11:'Non-Motor',
    12:'Light Rail',
    13:'Commuter Rail',
    15:'Ferry',
    16:'Passenger Only Ferry',
    17:'Connector',    # centroid connector
    18:'Connector',    # facility connector
    19:'HOV',    # HOV Only Freeway
    20:'HOV'    # HOV Flag
    }

gdf = gdf[gdf['FacilityType'].isin([5,6,7,8,9,11])]

In [55]:
len(gdf)

230054

In [56]:
# Intersect with urban growth area
gdf_growth_area = eg_conn.read_geolayer('urban_growth_area')
gdf_growth_area = gdf_growth_area[['county_name','geometry']].to_crs('EPSG:2285')

gdf = gpd.sjoin(gdf, gdf_growth_area, how="left")

gdf['rural'] = 'not_rural'
gdf.loc[gdf['county_name'].isnull(),'rural'] = 'rural'

In [57]:
gdf['IJBikeType'].value_counts()

0.0    170873
6.0     38230
1.0      7122
3.0      6707
8.0      4770
4.0      1907
2.0       481
9.0        51
5.0        12
Name: IJBikeType, dtype: int64

In [68]:
# Define transRefEdges facility types to match the study definitions

bike_type_map = {
    0: 'NoBikeFacility',    # No Bike Lane
    1: 'BikeLane',    # Striped Bike Lane
    2: 'ProtectedBikeLane',    # Protected Bike Lane
    3: 'NoBikeFacility',    # Paved Shoulder
    4: 'Sharrows',    # Shared Lane Markings
    5: 'NoBikeFacility',    # Bike Provision Undefined
    6: 'NoBikeFacility',    # Bike Provision Undefined
    8: 'SharedUsePath',    # Shared Use Path
    9: 'BufferedBikeLane',    # Buffered Bike Lane 
    10: 'NoBikeFacility',     # Neighborhood Greenway
    11: 'SharedUsePath'    # Shared use path
    
}

# Create a mapping of local vs other area type
area_type_map = {
    0: 'Other',
    1: 'Neighborhood'
}

# Get area type to be able to classify local streets
gdf['AreaType'] = gdf['LocalStreet'].map(area_type_map)

# We will follow this designation for any local street with speeds 30 mph or less and 2 lanes (1 in each direction)
# If speed limit > 30, set area type to non-neighborhood
gdf.loc[(gdf['SpeedLimit_new']>30, 'AreaType')] = 'Other'

# If number of lanes > 2, set area type to non-neighborhood regardless of street type
gdf.loc[(gdf['lanes_new']>2, 'AreaType')] = 'Other'

gdf_new = gdf.copy()
for dir in ['IJ','JI']:
    # Create new fields that use the bike facility
    gdf_new[dir+'BikeFacility'] = gdf_new[dir+'BikeType'].map(bike_type_map)

    # If in rural area, treat paved shoulder as bike lanes
    gdf_new.loc[((gdf_new['rural']=='rural')&(gdf_new[dir+'BikeType']==3)), dir+'BikeFacility'] = 'BikeLane'

    # Merge to df_data to get stress values
    gdf_new = gdf_new.merge(df_data, left_on=['SpeedLimit_new', 'lanes_new', dir+'BikeFacility','AreaType'], 
            right_on=['SpeedLimit', 'Lanes', 'BikeFacility', 'AreaType'], how='left', suffixes=['_IJ','_JI'])

In [72]:
gdf_new.head(20).to_clipboard()

In [60]:
# add impact of slope
# gdf.loc[(gdf['upslp'] > .02) & (gdf['upslp'] <= .04), 'Factor'] = gdf['Factor'] + 0.37
# gdf.loc[(gdf['upslp'] > .04) & (gdf['upslp'] <= .06), 'Factor'] = gdf['Factor'] + 1.2
# gdf.loc[(gdf['upslp'] > .06), 'Factor'] = gdf['Factor'] + 3.2

In [71]:
# Assign LTS based on stress factor
for dir in ['IJ','JI']:
    gdf_new.loc[gdf_new['Factor_'+dir] < 0.1, 'LTS_'+dir] = 1
    gdf_new.loc[(gdf_new['Factor_'+dir] >= 0.1) & (gdf_new['Factor_'+dir] < 0.3), 'LTS_'+dir] = 2
    gdf_new.loc[(gdf_new['Factor_'+dir] >= 0.3) & (gdf_new['Factor_'+dir] < 0.6), 'LTS_'+dir] = 3
    gdf_new.loc[gdf_new['Factor_'+dir] >= 0.6, 'LTS_'+dir] = 4

In [74]:
gdf_new[gdf_new['Factor_IJ'] != gdf_new['Factor_JI']].to_clipboard()

In [ ]:
# Create new column names because ArcGIS is a dinosaur that can't handle more than 10 character headers
# gdf.rename(columns={'BikeFacility':'bkfac', 'SpeedLimit':'speed'}, inplace=True)

In [1]:
gdf_new

NameError: name 'gdf_new' is not defined